In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import pandas as pd
import math

In [13]:
data = pd.read_csv('../1.data-preparation/new-data/data_Y1.csv')
data.describe()

,MemberID,LabCount_total,LabCount_months,DrugCount_total,DrugCount_months,no_Claims,no_Providers,no_Vendors,no_PCPs,no_PlaceSvcs,...,age_55,age_65,age_75,age_85,age_MISS,SexMALE,sexFEMALE,sexMISS,ClaimsTruncated,TARGET
count,7.603800e+04,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,...,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000,76038.000000
mean,4.994761e+07,6.148886,1.580289,10.118296,3.703661,11.320221,4.404127,3.705805,1.022344,2.145335,...,0.112365,0.116152,0.161130,0.085484,0.059733,0.367803,0.449025,0.183172,0.052224,0.467095
std,2.890145e+07,7.233436,1.814161,15.353455,4.089925,10.480493,3.341382,2.615952,0.150621,1.021198,...,0.315817,0.320410,0.367653,0.279602,0.236994,0.482211,0.497398,0.386810,0.222480,1.612194
min,2.100000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.483719e+07,0.000000,0.000000,0.000000,0.000000,3.000000,2.000000,2.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.990721e+07,4.000000,1.000000,3.000000,2.000000,8.000000,3.000000,3.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.510112e+07,10.000000,2.000000,14.000000,7.000000,16.000000,6.000000,5.000000,1.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,9.999863e+07,89.000000,12.000000,84.000000,12.000000,43.000000,27.000000,19.000000,3.000000,8.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,15.000000


In [14]:
data.drop("MemberID", axis = 1, inplace = True)
data.drop("Year", axis = 1, inplace = True)

y = data['TARGET']
X = data.loc[:, data.columns != 'TARGET']
X.head()

,LabCount_total,LabCount_months,DrugCount_total,DrugCount_months,no_Claims,no_Providers,no_Vendors,no_PCPs,no_PlaceSvcs,no_Specialities,...,age_45,age_55,age_65,age_75,age_85,age_MISS,SexMALE,sexFEMALE,sexMISS,ClaimsTruncated
0,2.0,1.0,5.0,3.0,8,4,4,2,3,3,...,0,0,0,0,0,0,0,0,1,0
1,10.0,1.0,30.0,10.0,13,7,7,1,3,4,...,0,0,0,0,0,1,0,1,0,0
2,3.0,1.0,0.0,0.0,6,3,3,1,2,3,...,0,0,0,0,0,0,0,1,0,0
3,2.0,1.0,1.0,1.0,2,2,2,1,2,2,...,0,0,0,0,0,0,0,0,1,0
4,2.0,1.0,6.0,5.0,6,5,4,2,2,4,...,0,0,0,0,0,0,0,1,0,0


In [15]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [17]:
def eval(model, X_test, y_test):
    pred_y = model.predict(X_test) # your predicted y values
    # The root mean square logarithmic error
    msle = np.mean( (np.log(pred_y + 1) - np.log(y_test + 1)) ** 2)
    rmsle = math.sqrt(msle)
    print ("RMSLE: %.4f" % rmsle)
    from sklearn.metrics import r2_score
    r2 = r2_score(pred_y, y_test)
    print ("R2 value: %.4f" % r2)

In [28]:
model = linear_model.LogisticRegressionCV(cv=5, tol=0.001, max_iter=10000,random_state=0, solver='lbfgs',
...                          multi_class='multinomial')
model.fit(X_train, y_train)
eval(model, X_test, y_test)

RMSLE: 0.5125
R2 value: 0.0000


In [29]:
data_submit = pd.read_csv('../1.data-preparation/new-data/data_Y2.csv')
data_submit.describe()

,MemberID,LabCount_total,LabCount_months,DrugCount_total,DrugCount_months,no_Claims,no_Providers,no_Vendors,no_PCPs,no_PlaceSvcs,...,age_55,age_65,age_75,age_85,age_MISS,SexMALE,sexFEMALE,sexMISS,ClaimsTruncated,TARGET
count,7.143500e+04,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000,...,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000
mean,4.993986e+07,6.650157,1.713670,10.719073,3.864030,12.512158,4.666046,3.912312,1.032617,2.216407,...,0.116232,0.120473,0.159460,0.075425,0.064688,0.357696,0.444180,0.198124,0.056611,0.437895
std,2.894077e+07,7.875864,1.975234,16.018236,4.210155,11.762587,3.637967,2.795337,0.181531,1.061151,...,0.320505,0.325516,0.366107,0.264078,0.245977,0.479325,0.496878,0.398589,0.231099,1.531780
min,4.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.477813e+07,0.000000,0.000000,0.000000,0.000000,4.000000,2.000000,2.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.995379e+07,5.000000,1.000000,3.000000,2.000000,8.000000,4.000000,3.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.521986e+07,10.000000,2.000000,15.000000,8.000000,17.000000,6.000000,5.000000,1.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,9.999882e+07,94.000000,12.000000,84.000000,12.000000,43.000000,30.000000,21.000000,4.000000,8.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,15.000000


In [30]:
data_submit.drop("MemberID", axis = 1, inplace = True)
data_submit.drop("Year", axis = 1, inplace = True)

y_submit = data_submit['TARGET']
X_submit = data_submit.loc[:, data_submit.columns != 'TARGET']
y_submit.head()

0    0
1    0
2    0
3    0
4    0
Name: TARGET, dtype: int64

In [31]:
X_submit = scaler.transform(X_submit)

1st time: 0.5075

2nd time: 0.5079

3rd time: 0.5065

In [32]:
eval(model, X_submit, y_submit)

RMSLE: 0.5065
R2 value: 0.0000
